In [ ]:
import xgboost as xgb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from sklearn.datasets import make_moons
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import time
import pickle
import imageio  # for animation
try:
    os.mkdir('./figures')
except:
    pass

In [ ]:
def make_data(n_samples,noise,shift):
    data,label = make_moons(n_samples=n_samples,noise=noise)
    x = data[:,0]
    y = data[:,1]

    df = pd.DataFrame(columns=['x','y','label'])
    df['x'] = data[:,0]
    df['y'] = data[:,1]
    df['label'] = label
    
    df['x'] = np.where(df['label'] == 0,df['x'],df['x'] + shift)
    
    return df

In [ ]:
df = make_data(n_samples=1000,noise=0.1,shift=0.0)
plt.scatter(df['x'],df['y'])
plt.show()

lr, knn, dt, svm, nb

In [ ]:
xgbc=xgb.XGBClassifier()

In [ ]:
figpath = './figures/xgboost'
try:
    os.mkdir(figpath)
except:
    pass

colors = np.array(['orange','blue'])
xlim = [-2,5];   x_min,x_max = xlim
ylim = [-1,1.5]; y_min,y_max = ylim
step = 0.02
xx, yy = np.meshgrid(np.arange(x_min, x_max+step, step), \
                     np.arange(y_min, y_max+step, step))
cm = plt.cm.RdBu

for i,s in enumerate(np.linspace(2,0,80)):
    df = make_data(n_samples=2000,noise=0.1,shift=s)
    Xtrain, Xtest, ytrain, ytest = train_test_split(df[['x','y']],
                                                    df['label'],
                                                    test_size = 0.2)
    figname = './figures/xgboost/'+str(10+i)+'.png'
    name = 'XGBoost'
    
    # fit
    t0 = time.time()
    xgbc.fit(Xtrain,ytrain)
    tfit = time.time()-t0
    # predict
    t0 = time.time()
    yp = xgbc.predict(Xtest)
    tpred = time.time()-t0
    # size
    pickle.dump(xgbc,open('size.pkl','wb'))
    kb = int(os.path.getsize('size.pkl'))
    os.remove('size.pkl')

    # Mismatches red
    match = np.where(yp!=ytest.values,0,1)

    accuracy = accuracy_score(yp,ytest)

    plt.figure(0,figsize=(4,4))
    plt_data = pd.DataFrame(columns=['x','y','color','marker'])
    plt_data['x'] = Xtest['x']
    plt_data['y'] = Xtest['y']
    plt_data['color'] = yp
    plt_data['marker'] = match

    # contour plot
    if hasattr(xgbc, "decision_function"):
        z = xgbc.decision_function(np.c_[xx.ravel(), yy.ravel()])
    else:
        z = xgbc.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
    z = z.reshape(xx.shape)
    plt.contourf(xx, yy, z, cmap=cm, alpha=0.7)        

    plt.scatter(plt_data[plt_data['marker']==1]['x'],
                plt_data[plt_data['marker']==1]['y'],
                color = colors[plt_data[plt_data['marker']==1]['color']],
                marker = '.'
               )

    plt.scatter(plt_data[plt_data['marker']==0]['x'],
                plt_data[plt_data['marker']==0]['y'],
                color = colors[plt_data[plt_data['marker']==0]['color']],
                edgecolors='k',
                marker = 'x'
               )

    plt.text(1.8,1.35,'Train: %.4f s'%(tfit))
    plt.text(1.8,1.20,'Predict: %.4f s'%(tpred))
    if kb<1024:
        plt.text(1.8,1.05,'Size: %.0f Bytes'%(kb))
    else:
        plt.text(1.8,1.05,'Size: %.0f kB'%(kb/1024.0))            
    plt.text(1.8,0.90,'Accuracy: %.0f%%'%(accuracy*100))
    plt.xlim([-2,5])
    plt.ylim([-1,1.5])
    plt.title(name)
    plt.tight_layout()
    plt.savefig(figname,dpi=300)
    plt.close()

In [ ]:
n = 80
images = []
for i in range(n):
    filename = './figures/xgboost/'+str(10+i)+'.png'
    images.append(imageio.imread(filename))
if images!=[]:
    imageio.mimsave('./figures/xgboost.mp4', images)